In [1]:
import json
import random
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score


In [2]:
import sys, os, json, random, logging, time, socket
import numpy as np
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import gridspec
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from matplotlib.backends.backend_pdf import PdfPages
from scipy import interp
from sklearn.preprocessing import MinMaxScaler
from sklearn import manifold
import matplotlib.pylab as pylab
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [3]:
malware_file = 'amd_malware_metadata/malware_post_pca_data.txt'

with open(malware_file, 'r') as data_file:
    mlw_data = json.load(data_file)

In [4]:
malware = {}
for m in mlw_data.keys():
    for each_sample in mlw_data[m]:
        if m not in malware:
            malware[m] = np.array(each_sample)
        else:
            malware[m] = np.vstack((malware[m], each_sample))


In [5]:
malware_fam = {}
with open('fam_data/myth_label.txt', 'r') as jk:
    for line in jk:
        l = line.strip().split("\t")
        if l[1].startswith("SINGLETON:"):
            l[1] = l[1].split(":")[0]
        if l[0] in malware:
            if l[1] not in malware_fam.keys():
                malware_fam[l[1]] = malware[l[0]]
            else:
                malware_fam[l[1]] = np.vstack(
                    (malware_fam[l[1]], malware[l[0]]))


In [6]:
tt = []
with open('fam_data/myth_label.txt', 'r') as jk:
    for line in jk:
        l = line.strip().split("\t")
        if l[1].startswith("SINGLETON:"):
            l[1] = l[1].split(":")[0]
        if l[1] not in tt:
            tt.append(l[1])

In [73]:
len(malware_fam.keys())

40

# benign

In [7]:
benign_files = [\
        'amd_desktop_metadata/benign_post_pca_data.txt',\
        'amd_python_benign_metadata/python_benign_post_pca_data.txt'
        ]
malware_files = ['amd_malware_metadata/malware_post_pca_data.txt']

benign_data = []
for files in benign_files:
    with open(files, 'r') as data_file:
        json_data = json.load(data_file)

    for bm in json_data.keys():
        for each_sample in json_data[bm]:
            if benign_data == []:
                benign_data = np.array(each_sample) 
            else:
                benign_data = np.vstack((benign_data, each_sample))

    data_file.close() 


/tmp/ipykernel_99792/236228987.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if benign_data == []:


In [8]:
benign_data.shape

(45648, 12)

In [15]:
class learnAll:
    def __init__(self, malware, benign):
        self.malware_data = malware
        self.benign_data = benign

        malware_label = np.ones(len(malware))
        benign_label = np.zeros(len(benign_data))
        sample_size = min(len(benign_label), len(malware_label))
        self.data = np.vstack((
            random.sample(list(benign_data), sample_size),
            random.sample(list(malware), sample_size)
        ))
        self.label = np.concatenate((
            random.sample(list(benign_label), sample_size),
            random.sample(list(malware_label), sample_size)
        ))

    def setup_train_test_split(self):
        """
        Setup_train_test_split uses default split (scikit-learn split) to separate the 
        training and testing dataset.
        """
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.data, self.label, test_size=0.1,
            random_state=random.randint(0, 2**32 - 1))

    def setup_classifiers(self):
        """
        setup_classifiers only generate the classifier class objects. nothing more.
        """
        self.names = [
            "Nearest Neighbors", "Decision Tree",
            "Random Forest",
            "AdaBoost", "Naive Bayes",
            "Neural Net" \
            # "Logistic Regression",\
            # "Linear SVM", "Rbf SVM", "Poly SVM", "Sigmoid SVM"\
            # , "Gaussian Process"
        ]
        max_iterations = 1000
        self.classifiers = [
            KNeighborsClassifier(
                n_neighbors=5, weights='uniform', algorithm='auto', n_jobs=-1),
            DecisionTreeClassifier(
                max_depth=100, min_samples_split=12, min_samples_leaf=12,
                max_features=None,
                random_state=int(round(time.time()))),
            RandomForestClassifier(max_depth=100, min_samples_split=12,
                                   min_samples_leaf=12,
                                   n_estimators=100, max_features=None,
                                   random_state=int(round(time.time()))),
            AdaBoostClassifier(algorithm='SAMME.R', n_estimators=200,
                               random_state=int(round(time.time()))),
            GaussianNB(priors=[0.5, 0.5]),
            MLPClassifier(hidden_layer_sizes=(100, 100, 100, 100),
                          alpha=100, solver='lbfgs',
                          max_iter=max_iterations,
                          activation='tanh', tol=1e-5,
                          warm_start=True) \
            # LogisticRegression(penalty='l2', tol=1e-4, C=1e2,\
            # fit_intercept=True, solver='lbfgs', \
            # class_weight='balanced', max_iter=max_iterations), \
            # SVC(kernel="linear", C=1e2, tol=1e-4, max_iter=max_iterations,\
            # probability= True),\
            # SVC(kernel="rbf", C=1e2, tol=1e-4, max_iter=max_iterations,\
            # probability=True, shrinking=True),
            # SVC(kernel="poly", C=1e2, degree=4, tol=1e-4,\
            # max_iter=max_iterations, probability=True),\
            # SVC(kernel="sigmoid", C=1e2, gamma=1e-1, tol=1e-3, \
            # max_iter=max_iterations, probability=True, \
            # shrinking=True)#,\
            # GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1, \
            # copy_X_train=False, \
            # max_iter_predict=100, warm_start=False )\
        ]
        # self.classifiers = [ \
        #                 LogisticRegression(penalty='l2', tol=1e-9, C=1,\
        #                   fit_intercept=True, solver='lbfgs', \
        #                   class_weight='balanced', max_iter=max_iterations*100, n_jobs=-1) \
        #                 ]

    def text_class_report(self, names,fam):
        """
        Text_class_report for experiment log files. text report updates the self.report 
        to log the results in the terminal. The results include precision, recall, f1_score
        and support. The results are dumped into classification report. The labels are 0 and 1.
        """
        # results
        precision, recall, f1_score, support = \
            precision_recall_fscore_support(self.y_test, self.prediction,
                                            beta=1.0, average='binary', labels=[0, 1], pos_label=0)
        self.report[names] = {}
        self.report[names]['Accuracy'] = accuracy_score(self.y_test, self.prediction)
        self.report[names]['precision'] = precision
        self.report[names]['recall'] = recall
        self.report[names]['f1_score'] = f1_score
        self.report[names]['support'] = support
        self.report[names]['param'] = \
            self.classifiers[self.names.index(names)].get_params()

        with open(fam+'_classification_report.txt', 'w') as outfile:
            json.dump(self.report, outfile, indent=4)
        outfile.close()
        # print(self.report)

    def roc_curve_report(self, name):
        """
        Use fpr and tpr information to record the roc information.
        """
        fpr, tpr, _ = roc_curve(
            self.y_test, self.predict_proba[:, 0], pos_label=0)
        roc_auc = auc(fpr, tpr)
        self.report[name]['fpr'] = list(fpr)
        self.report[name]['tpr'] = list(tpr)
        self.report[name]['roc_auc'] = roc_auc

        with open('classification_report.txt', 'w') as outfile:
            json.dump(self.report, outfile, indent=2)
        outfile.close()

    def run(self,fam):
        """
        run function has the minimum experimental setup. It uses the basic training and 
        testing split.
        """

        self.setup_train_test_split()
        self.setup_classifiers()
        self.report = {}
        for names, clf in zip(self.names, self.classifiers):
            print(names + " in process ....")
            # train
            clf.fit(self.X_train, self.y_train)
            # if names == 'Decision Tree':
            #     tree.export_graphviz(clf, out_file=self.result_path + 'tree.dot')
            # test

            self.prediction = clf.predict(self.X_test)
            self.text_class_report(names,fam)

            self.predict_proba = clf.predict_proba(self.X_test)
            self.roc_curve_report(names)

            print(names + " testing completed!")

    def cv_text_class_report(self, names):
        """
        cv_text_class_report generates logs for cross validation results.
        """
        chunk_length = len(self.label) / 10
        self.report[names] = {}
        for idx in range(10):
            precision, recall, f1_score, support = \
                precision_recall_fscore_support(
                    self.label[idx * chunk_length: (idx + 1) * chunk_length],
                    self.prediction[idx *
                                    chunk_length: (idx + 1) * chunk_length],
                    beta=1.0, average='binary', labels=[0, 1], pos_label=0)
            self.report[names][idx] = {}
            self.report[names][idx]['precision'] = precision
            self.report[names][idx]['recall'] = recall
            self.report[names][idx]['f1_score'] = f1_score
            self.report[names][idx]['support'] = support

        # with open(self.result_path + 'cv_classification_report.txt', 'w') as outfile:
        #     json.dump(self.report, outfile, indent=2)
        # outfile.close()
        print(self.report)

    def classic_cross_validation(self):
        """
        classic_cross_validation runs cross validation with ALL the data in the feature
        vectors. It does NOT separate feature vectors between training and testing dataset.
        """
        print('Start to cross validate ...')
        self.setup_classifiers()
        self.report = {}
        for names, clf in zip(self.names, self.classifiers):
            self.logger.info(names + " in process ....")
            # test
            zipped_list = zip(self.data, self.label)
            random.shuffle(zipped_list)
            self.data, self.label = zip(*zipped_list)
            self.data = list(self.data)
            self.label = list(self.label)
            self.prediction = cross_val_predict(
                clf, self.data, self.label, cv=10)
            self.cv_text_class_report(names)

            print(names + " testing completed!")


In [10]:
learn = learnAll(malware_fam['SINGLETON'], benign_data)


In [11]:
learn.run()

Nearest Neighbors in process ....
Nearest Neighbors testing completed!
Decision Tree in process ....
Decision Tree testing completed!
Random Forest in process ....
Random Forest testing completed!
AdaBoost in process ....
AdaBoost testing completed!
Naive Bayes in process ....
Naive Bayes testing completed!
Neural Net in process ....
Neural Net testing completed!


In [56]:
# learn.cv_text_class_report()

In [16]:
for fam in malware_fam:
    if len(malware_fam[fam]) > 500:
        print(len(malware_fam[fam]))
        print(fam)
        learn = learnAll(malware_fam[fam], benign_data)
        learn.run(fam)


23300
vobfus
Nearest Neighbors in process ....
Nearest Neighbors testing completed!
Decision Tree in process ....
Decision Tree testing completed!
Random Forest in process ....
Random Forest testing completed!
AdaBoost in process ....
AdaBoost testing completed!
Naive Bayes in process ....
Naive Bayes testing completed!
Neural Net in process ....
Neural Net testing completed!
1276
SINGLETON
Nearest Neighbors in process ....
Nearest Neighbors testing completed!
Decision Tree in process ....
Decision Tree testing completed!
Random Forest in process ....
Random Forest testing completed!
AdaBoost in process ....
AdaBoost testing completed!
Naive Bayes in process ....
Naive Bayes testing completed!
Neural Net in process ....
Neural Net testing completed!
1593
beebone
Nearest Neighbors in process ....
Nearest Neighbors testing completed!
Decision Tree in process ....
Decision Tree testing completed!
Random Forest in process ....
Random Forest testing completed!
AdaBoost in process ....
AdaBo

/home/null_void/Music/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Neural Net testing completed!
762
zbot
Nearest Neighbors in process ....
Nearest Neighbors testing completed!
Decision Tree in process ....
Decision Tree testing completed!
Random Forest in process ....
Random Forest testing completed!
AdaBoost in process ....
AdaBoost testing completed!
Naive Bayes in process ....
Naive Bayes testing completed!
Neural Net in process ....
Neural Net testing completed!


/home/null_void/Music/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [15]:
malware_file = 'amd_malware_metadata/malware_post_pca_data.txt'

with open(malware_file, 'r') as data_file:
    mlw_data = json.load(data_file)

/tmp/ipykernel_61967/2171748639.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.benign_data == []:
/tmp/ipykernel_61967/2171748639.py:46: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.malware_data == []:


In [ ]:
class SpecilizedHMD:
	def __init__ (self, logger, data_path_prefix='./'):
		"""
		init function for initiating the members
		"""

		self.benign_data    = []
		self.malware_data   = []
		self.benign_dict    = {}
		self.malware_dict   = {}
		self.result_path    = './'

		# setup seed
		random.seed(os.urandom(100))

